In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,45782
2,Huelva,Confirmados PDIA 14 días,270
3,Huelva,Tasa PDIA 14 días,"52,61414346123117"
4,Huelva,Confirmados PDIA 7 días,113
5,Huelva,Tasa PDIA 7 dias,"22,019993374515266"
6,Huelva,Total Confirmados,45985
7,Huelva,Curados,44870
8,Huelva,Fallecidos,418


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  45782.0


/tmp/ipykernel_62415/1396922249.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  12601.0


/tmp/ipykernel_62415/2509482988.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

/tmp/ipykernel_62415/2690025379.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
/tmp/ipykernel_62415/2690025379.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
/tmp/ipykernel_62415/2690025379.py:3: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]


In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 6849 personas en los últimos 7 días 

Un positivo PCR cada 3887 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,45782.0,113.0,270.0,513170.0,22.019993,52.614143,16.0
Huelva-Costa,27081.0,61.0,137.0,289548.0,21.067319,47.315126,10.0
Condado-Campiña,14236.0,43.0,108.0,156231.0,27.523347,69.128406,8.0
Cartaya,2052.0,21.0,52.0,20083.0,104.566051,258.925459,5.0
Moguer,1994.0,13.0,27.0,21867.0,59.450313,123.473728,3.0
Huelva (capital),12601.0,21.0,37.0,143837.0,14.599860,25.723562,2.0
Lepe,3030.0,5.0,11.0,27880.0,17.934003,39.454806,2.0
Almonte,2309.0,7.0,14.0,24507.0,28.563268,57.126535,1.0
Ayamonte,2091.0,3.0,8.0,21104.0,14.215315,37.907506,1.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() != 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villarrasa,232.0,5.0,7.0,2211.0,226.142017,316.598824,1.0
Cartaya,2052.0,21.0,52.0,20083.0,104.566051,258.925459,5.0
Zalamea la Real,119.0,5.0,5.0,3054.0,163.719712,163.719712,0.0
Palos de la Frontera,1050.0,7.0,18.0,11742.0,59.615057,153.295861,1.0
Moguer,1994.0,13.0,27.0,21867.0,59.450313,123.473728,3.0
Palma del Condado (La),1426.0,4.0,10.0,10801.0,37.033608,92.584020,1.0
Valverde del Camino,858.0,1.0,10.0,12750.0,7.843137,78.431373,0.0
Condado-Campiña,14236.0,43.0,108.0,156231.0,27.523347,69.128406,8.0
Aroche,130.0,2.0,2.0,3054.0,65.487885,65.487885,0.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Valverde del Camino,858.0,1.0,10.0,12750.0,7.843137,78.431373,0.0,0.100000
Punta Umbría,1275.0,3.0,10.0,15355.0,19.537610,65.125366,0.0,0.300000
Ayamonte,2091.0,3.0,8.0,21104.0,14.215315,37.907506,1.0,0.375000
Sierra de Huelva-Andévalo Central,4039.0,9.0,24.0,67391.0,13.354899,35.613064,0.0,0.375000
Palos de la Frontera,1050.0,7.0,18.0,11742.0,59.615057,153.295861,1.0,0.388889
Condado-Campiña,14236.0,43.0,108.0,156231.0,27.523347,69.128406,8.0,0.398148
Palma del Condado (La),1426.0,4.0,10.0,10801.0,37.033608,92.584020,1.0,0.400000
Cartaya,2052.0,21.0,52.0,20083.0,104.566051,258.925459,5.0,0.403846
Huelva,45782.0,113.0,270.0,513170.0,22.019993,52.614143,16.0,0.418519
